# 导入函数库

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras  # 在TF2.0版本中,已经是自带Keras了,所以不需要额外安装
import tensorflow_model_optimization as tfmot  # 导入TF2.0的模型优化函数库,降低模型优化难度,相当于调用借口解决问题
import zipfile
import tempfile
import tensorflow_datasets as tfds # 这个是之前说过的Tensorflow Datasets

# 如果出现显存不够的错误,把这个代码加上

可以展示下不加这个出现错误的情形

In [2]:
# 加入下面这个代码片段主要是因为TF2.0对RTX20系列显卡支持不是很好,容易爆显存,所以设置成用多少占多少显存,而不是一次性全占了(默认)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# 定义网络结构

## 一些参数设置

In [3]:
layers = tf.keras.layers
models = tf.keras.models
keras_utils = tf.keras.utils

In [4]:
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
                       'releases/download/v0.1/'
                       'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')  # 此处的预训练模型不带全连接层

## VGG16模型改写用于Cifar10分类

In [5]:
# 关于这里把官方VGG16函数拿了出来,原因是这样的,我们知道Keras主要有Sequential model和Functional model两种构建模型方式,
# 而在进行迁移学习的时候，我们经常是加载预训练的模型(设置include_top=False),此时我们很常见的做法是add layer到模型上,
# 这样会形成模型嵌套,这个在使用global_pruning的时候会出错,所以我把VGG16函数拿了出来,手动改了下
def VGG16(include_top=False,
         weights='imagenet',
         input_shape=(32, 32, 3),
         pooling='avg',
         classes=10,
         **kwargs):
    img_input = layers.Input(shape=input_shape)  # 输入节点
    
    # Block 1
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1')(img_input)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    if pooling == 'avg':
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(128, activation='relu', name='fc1')(x)
        x = layers.Dense(classes, activation='softmax', name='predictions')(x) # 此处是10分类，而不是ImageNet的1000分类
    
    inputs = img_input  # inputs是输入节点, x是输出节点
    model = models.Model(inputs, x , name='vgg16')  # 生成一个Model, 需要指定输入和输出
    
    weights_path = keras_utils.get_file(
        'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',  # 此处加载的是不存在全连接层的预训练模型
        WEIGHTS_PATH_NO_TOP,
        cache_subdir='models',
        file_hash='6d6bbae143d832006294945121d1f1fc')
    model.load_weights(weights_path, by_name=True)  
    # 加载在ImageNet上预训练过的模型，注意by_name参数很有用，把layer和layer name对应上了
    
    return model

## 声明一个VGG16模型实例

In [6]:
model = VGG16()
model.load_weights('./baseline_finetune.h5')

## 这个函数可以用于查看网络结构和参数量

In [7]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

## 对特征提取层进行冻结,加快模型训练速度

可以展示下不加的情形，更直观的说明为什么需要训练更长时间

In [8]:
length = len(model.layers)  # 查看模型有多少层Layer
print(length)

22


In [9]:
for i in range(length):
    print(model.layers[i].trainable)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


## 进行模型训练

In [10]:
def convert(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32) # Cast and normalize the image to [0,1]
    return image, label

def augment(image, label):
    image, label = convert(image, label)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.resize_with_crop_or_pad(image, 38, 38) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[32, 32, 3]) # Random crop back to 32x32
    image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness

    return image, label

In [11]:
# 此处使用tfds的方式构建data pipeline
(raw_test, raw_train), metadata = tfds.load(
    'cifar10',
    split=['test', 'train'],
    with_info=True,
    as_supervised=True
)

IMG_SIZE = 32 # All images will be resized to 32X32

BATCH_SIZE = 256
SHUFFLE_BUFFER_SIZE = 50000

# 可以体验下这里是否加prefetch(tf.data.experimental.AUTOTUNE)和cache()的区别，对训练速度，以及CPU负载有影响
train_batches = raw_train.shuffle(SHUFFLE_BUFFER_SIZE, reshuffle_each_iteration=True).map(augment).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_batches = raw_test.map(convert).batch(BATCH_SIZE)

# compile模型并训练
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])

## Baseline的test acc和推断速度

In [12]:
# 此处我们可以看到Baseline的test acc和inference速度为6ms/step
_, baseline_model_accuracy = model.evaluate(test_batches, verbose=1)
print('Baseline test accuracy: ', baseline_model_accuracy)

keras_file = './test.h5'
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to: ', keras_file)

40/40 [==============================] - 1s 22ms/step - loss: 1.6255 - accuracy: 0.8348
Baseline test accuracy:  0.8348000049591064
Saved baseline model to:  ./test.h5


# 对Baseline进行quantization

In [13]:
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
quantize_layer (QuantizeLaye (None, 32, 32, 3)         3         
_________________________________________________________________
quant_block1_conv1 (Quantize (None, 32, 32, 64)        1923      
_________________________________________________________________
quant_block1_conv2 (Quantize (None, 32, 32, 64)        37059     
_________________________________________________________________
quant_block1_pool (QuantizeW (None, 16, 16, 64)        1         
_________________________________________________________________
quant_block2_conv1 (Quantize (None, 16, 16, 128)       74115     
_________________________________________________________________
quant_block2_conv2 (Quantize (None, 16, 16, 128)       147843

## 训练Baseline_quantized

In [14]:
batch_size = BATCH_SIZE
epochs = 20

q_aware_model.fit(train_batches,
                     batch_size=batch_size, epochs=epochs)

Epoch 1/20
196/196 [==============================] - 31s 159ms/step - loss: 1.6934 - accuracy: 0.7798
Epoch 2/20
196/196 [==============================] - 31s 156ms/step - loss: 1.5625 - accuracy: 0.9037
Epoch 3/20
196/196 [==============================] - 31s 156ms/step - loss: 1.5249 - accuracy: 0.9391
Epoch 4/20
196/196 [==============================] - 32s 162ms/step - loss: 1.5128 - accuracy: 0.9502
Epoch 5/20
196/196 [==============================] - 30s 156ms/step - loss: 1.5065 - accuracy: 0.9560
Epoch 6/20
196/196 [==============================] - 31s 156ms/step - loss: 1.5049 - accuracy: 0.9574
Epoch 7/20
196/196 [==============================] - 31s 156ms/step - loss: 1.5049 - accuracy: 0.9573
Epoch 8/20
196/196 [==============================] - 32s 162ms/step - loss: 1.5039 - accuracy: 0.9584
Epoch 9/20
196/196 [==============================] - 31s 156ms/step - loss: 1.5031 - accuracy: 0.9590
Epoch 10/20
196/196 [==============================] - 31s 159ms/step - l

# 对比Baseline和quantized之后的test acc

In [15]:
_, baseline_model_accuracy = model.evaluate(
    test_batches, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   test_batches, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

Baseline test accuracy: 0.8348000049591064
Quant test accuracy: 0.8457000255584717


# Create quantized model for TFLite backend

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

# 保存int8的TFLite模型

In [17]:
tflite_model_quant_file = './int8.tflite'
with open(tflite_model_quant_file, 'wb') as f:
    f.write(quantized_tflite_model)

# 封装函数用于评估TFLite模型

In [18]:
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    labels = []
    for i, (image, label) in enumerate(raw_test):
        if i % 1000 == 0:
          print('Evaluated on {n} results so far.'.format(n=i))
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        image, label = convert(image, label)
    #     print(label.numpy())
        test_image = np.expand_dims(image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)
        labels.append(label.numpy())

    print('\n')
    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == labels).mean()
    return accuracy

# 对比Quantized之后的TF和TFLite模型的test acc

In [19]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.8473
Quant TF test accuracy: 0.8457000255584717
